In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
#If you want the Keras modules you write to be compatible with both 
#Theano (th) and TensorFlow (tf), you have to write them via the abstract Keras backend API.
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras.models import Model
#Till we have more data, we use ceil() at validation steps
#import math

Using TensorFlow backend.


In [3]:
image_path="data"
IMG_SIZE=100

#https://keras.io/preprocessing/image/#imagedatagenerator-class
data_generator=ImageDataGenerator(validation_split=0.2)

train_data = data_generator.flow_from_directory(image_path,target_size=(100,100), color_mode='rgb', batch_size=10, subset='training')
valid_data = data_generator.flow_from_directory(image_path,target_size=(100,100), color_mode='rgb', batch_size=10, subset='validation')

STEP_SIZE = train_data.n//train_data.batch_size
VALIDATION_STEPS = valid_data.n // valid_data.batch_size

print("Classes : ",train_data.class_indices)
print("VALIDATION_STEPS : ",VALIDATION_STEPS)

Found 1087 images belonging to 2 classes.
Found 270 images belonging to 2 classes.
Classes :  {'Face_Images': 0, 'Mobile_Images': 1}
VALIDATION_STEPS :  27


In [4]:
base_model=ResNet50(include_top=False, weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)

#we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x)
#x=Dropout(0.3)(x)
x=Dense(512,activation='relu')(x)
#x = Dense(256,activation='relu')(x)
#final layer with softmax activation
preds=Dense(2,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)
#model.summary()
print(len(model.layers))

W0926 16:02:53.134119 20208 deprecation_wrapper.py:119] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0926 16:02:53.148130 20208 deprecation_wrapper.py:119] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0926 16:02:53.154135 20208 deprecation_wrapper.py:119] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0926 16:02:53.182160 20208 deprecation_wrapper.py:119] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0926 16:02:5

179


In [5]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])
history = model.fit_generator(generator=train_data,
                   steps_per_epoch=STEP_SIZE,
                   validation_data=valid_data,
                   validation_steps=VALIDATION_STEPS,
                   epochs=10)

print(history.history.keys())

W0926 16:03:01.835018 20208 deprecation_wrapper.py:119] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0926 16:03:01.977148 20208 deprecation.py:323] From C:\Users\sidha\.conda\envs\Keras\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1/convolution}}]]
	 [[loss/mul/_3665]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1/convolution}}]]
0 successful operations.
0 derived errors ignored.

In [1]:
#Plotting graphs for better viewing the model
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.save("resnet.h5")
model.save_weights('my_model_weights.h5')
model.summary()

In [ ]:
from keras.preprocessing import image
import numpy as np
import keras

#grayscale=True can also be passed
test_image = image.load_img("sida.jpeg",color_mode='rgb', target_size = (100, 100)) 
test_image = image.img_to_array(test_image)
#predict expects an array of images.
test_image = np.expand_dims(test_image, axis = 0)

p=model.predict(test_image)
print("The predicted array is :",p)
print("The prediction index is :",p.argmax(axis=-1))
print("The predicted class is :",keras.np_utils.probas_to_classes(p))

In [ ]:
from keras import metrics
cm = metrics.confusion_matrix(train_data.classes, y_pred)
# or
#cm = np.array([[1401,    0],[1112, 0]])

plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.xticks([], [])
plt.yticks([], [])
plt.title('Confusion matrix ')
plt.colorbar()
plt.show()